In [1]:
import numpy as np
import cv2
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

1 Implement the basic stereo algorithm of taking a window around every pixel in one image and search for the best match along the same scan line in the other image. You will do this both left to right and right to left. Remember: because of depth changes (discontinuities) some pixels visible in the left image are not in the right and vice a versa. So you will match in both directions.

    a) Implement the SSD match algorithm, and create a disparity image D(x,y) such that L(x,y) = R(x+D(x,y),y) when matching from left to right. Also match from right to left.
    Output: Show DL(x,y) [matching from left to right] and DR(x,y)[matching from right to left] as images. These disparity images may need to be scaled and shifted to display correctly. They should show a central square moved 2 pixels to the left or right.

In [2]:
img1L = cv2.imread("./Data/leftTest.png", cv2.COLOR_BGR2GRAY)
img1R = cv2.imread("./Data/rightTest.png", cv2.COLOR_BGR2GRAY)

In [146]:
def match_SSD(img1, img2, win_size=10, offset=100, l2r=True):
    img1_h, img1_w = img1.shape
    ing2_h, img2_w = img2.shape
    D = np.zeros((img1_h, img1_w))
    for ix in range(int(win_size/2), img1_w - int(win_size/2)):
        for iy in range(int(win_size/2), img1_h - int(win_size/2)):
            
            w1 = img1[iy - int(win_size/2) : iy + int(win_size/2), ix - int(win_size/2) : ix + int(win_size/2)]
            disparity = float('inf')

            for d in range(offset):
                if l2r:
                    w2 = img2[iy - int(win_size/2) : iy + int(win_size/2), ix - int(win_size/2) - d : ix + int(win_size/2) - d]
                else:
                    w2 = img2[iy - int(win_size/2) : iy + int(win_size/2), ix - int(win_size/2) + d : ix + int(win_size/2) + d]
                if w2.shape == w1.shape:
                    diff = np.subtract(w1, w2)
                    diff = diff.astype(int)
                    diff = np.power(diff, 2)
                    disp = np.sum(diff)
                    if disp < disparity:
                        disparity = disp
                        D[iy, ix] = d
    return D

In [148]:
def scale_d(D):
    m = np.max(D)
    return D/m * 255

In [149]:
D1L = match_SSD(img1L, img1R, 10, 10, True)
cv2.imwrite('ps2-1-a-1.png', scale_d(D1L))

True

In [36]:
D1R = match_SSD(img1R, img1L, 10, 10, False)
cv2.imwrite('ps2-1-a-2.png', scale_d(D1R))

True

2 Now we’re going to try this on a real image pair: proj2-pair1-L .png and proj2-pair1-R .png. Since these are color images create gray scale versions. You can use rgb2gray or your own function.

    a) Again apply your SSD match algorithm, and again create a disparity image D(x,y) such that L(x,y) = R(x+DL(x,y),y) when matching from left to right. Also match from right to left. 
    Output: Show DL(x,y) [matching from left to right] and DR(x,y)[matching from right to left] as images. These disparity images may need to be scaled and shifted to display correctly.

In [50]:
img2L = cv2.imread("./Data/proj2-pair1-L.png")
img2R = cv2.imread("./Data/proj2-pair1-R.png")

In [51]:
img2L = cv2.cvtColor(img2L, cv2.COLOR_BGR2GRAY)
img2R = cv2.cvtColor(img2R, cv2.COLOR_BGR2GRAY)

In [53]:
img2R.shape

(511, 640)

In [147]:
D2L = match_SSD(img2L, img2R, 10, 100, True)
cv2.imwrite('ps2-2-a-1.png', scale_d(D2L))

True

In [58]:
D2R = match_SSD(img2R, img2L, 10, 100, False)
cv2.imwrite('ps2-2-a-2.png', scale_d(D2R))

True

    b) Also in this directory are the ground truth disparity images proj2-pair1-Disp-L .png and proj2-pair1-Disp-R .png. Compare your results.
    Output: description of the differences between your results and the ground truth

Answer:
My output has lots of noise, especially when the original image has small details.

3 SSD is not very robust to certain perturbations. We’re going to try to see the effect of perturbations: 

    a. Add some Gaussian noise to the image; either one or both. Make the noise sigma big enough that you can tell some noise has been added. Run SSD again.
    Output: Disparity images and analysis of result compared to part 1.

In [123]:
mean = 0
sigma = 19
noise = np.random.normal(mean, sigma, img2L.shape)
img2L_noised = img2L.copy()
img2L_noised = img2L_noised + noise
img2L_noised = np.clip(img2L_noised, 0, 255)
cv2.imwrite('img2_noise.png', img2L_noised)

True

In [75]:
img2R_noised = img2R.copy()
img2R_noised = img2R_noised + noise
img2R_noised = np.clip(img2R_noised, 0, 255)

In [ ]:
D2L_noised = match_SSD(img2L_noised, img2R, 10, 100, True)
cv2.imwrite('ps2-3-a-1.png', scale_d(D2L_noised))

In [103]:
D2L_noised2 = match_SSD(img2L_noised, img2R_noised, 10, 100, True)
cv2.imwrite('ps2-3-a-2.png', scale_d(D2L_noised2))

True

In [106]:
D2L_noised3 = match_SSD(img2L, img2R_noised, 10, 100, True)
cv2.imwrite('ps2-3-a-3.png', scale_d(D2L_noised3))

True

Answer: After added noise to the left image, I found the result of disparity image looks closer to the real depth image. If there are similar patterns in an epipolar line, the disparity may be minimize in a wrong place, so we get a wrong depth image in some position. But after added noise to one image,  

If add noises to both two images, there is no pixel displacement for the noises as for the objects in the image between the left and right original image. So for many patterns, they cannot find a good result. F

    b. Instead of the Gaussian noise, increase the contrast (multiplication) of one of the images by just 10%. Run SSD again.
    Output: Disparity images and analysis of result compared to part 1.

In [101]:
img2L_contrast = np.zeros((img2L.shape))
for iy in range(img2L.shape[0]):
    for ix in range(img2L.shape[1]):
        if img2L[iy, ix] > 128:
            img2L_contrast[iy, ix] = 1.1 * img2L[iy, ix]
        else:
            img2L_contrast[iy, ix] = 0.9 * img2L[iy, ix]
        if img2L_contrast[iy, ix] > 255:
            img2L_contrast[iy, ix] = 255
        if img2L_contrast[iy, ix] < 0:
            img2L_contrast[iy, ix] = 0

In [105]:
D2L_contrast = match_SSD(img2L_contrast, img2R, 10, 100, True)
cv2.imwrite('ps2-3-b.png', scale_d(D2L_contrast))

True

Answer:
The contrast image made the depth image less noise, its better for small details because that can make it easier to match. But areas like the plaster head, there are many similar patterns in an epipolar line, so it calculated to a wrong depth comparing with other objects.

4 Now you’re going to use (not implement yourself unless you want) an improved method, called normalized correlation – this is discussed in the book. The basic idea is that we think of two image patches as vectors and to compute the angle between them – much like normalized dot products.

    a) Implement a window matching stereo algorithm using some form of normalized correlation. Test it on the original images both left to right and right to left.
    Output: disparity images and description of how it compares to the SSD version and to the ground truth

In [142]:
def match_norm(img1, img2, win_size=10, search_len=10, l2r=True):
    img1_h, img1_w = img1.shape
    img2_h, img2_w = img2.shape
    img1 = img1.astype(np.uint8)
    img2 = img2.astype(np.uint8)
    D = np.zeros((img1_h, img1_w))
    for ix in range(int(win_size/2), img1_w - int(win_size/2)):
        for iy in range(int(win_size/2), img1_h - int(win_size/2)):
            
            w1 = img1[iy - int(win_size/2) : iy + int(win_size/2), ix - int(win_size/2) : ix + int(win_size/2)]
            disparity = float('inf')
            #w1 = w1.astype(np.uint8)
            for d in range(search_len):
                if l2r:
                    w2 = img2[iy - int(win_size/2) : iy + int(win_size/2), ix - int(win_size/2) - d : ix + int(win_size/2) - d]
                else:
                    w2 = img2[iy - int(win_size/2) : iy + int(win_size/2), ix - int(win_size/2) + d : ix + int(win_size/2) + d]
                
                # w2 = w2.astype(np.uint8)
                if w2.shape == w1.shape:
                    disp = cv2.matchTemplate(w1, w2, method=cv2.TM_CCOEFF_NORMED)
                    if disp < disparity:
                        disparity = disp
                        D[iy, ix] = d
    return D

In [150]:
D2nL = match_norm(img2L, img2R, 20, 100, True)
cv2.imwrite('ps2-4-a-1.png', scale_d(D2nL))

True

In [151]:
D2nR = match_norm(img2R, img2L, 20, 100, False)
cv2.imwrite('ps2-4-a-2.png', scale_d(D2nR))

True

Answer:


    b) Now test it on both the Gaussian noise and contrast boosted versions from 2a and 2b. Output: Again disparity images and analysis of results.

In [125]:
print(img2L_noised.shape)
print(img2R.shape)

(511, 640)
(511, 640)


In [144]:
D2nL_noised = match_norm(img2L_noised, img2R, 10, 100, True)
cv2.imwrite('ps2-4-b-1.png', scale_d(D2nL_noised))

True

In [145]:
D2nL_contrast = match_norm(img2L_contrast, img2R, 10, 100, True)
cv2.imwrite('ps2-4-b-2.png', scale_d(D2nL_contrast))

True

Answer:


5 Finally, there is a second pair of images: proj2-pair2-L .png and proj2-pair2-R .png

    a. Try your algorithms on this pair. Play with the images – smooth, sharpen, etc. Keep comparing to the ground truth.
    Output: disparity images and analysis of what it takes to make stereo work using a window based approach.

In [152]:
img3L = cv2.imread("./Data/proj2-pair2-L.png")
img3R = cv2.imread("./Data/proj2-pair2-R.png")
img3L = cv2.cvtColor(img3L, cv2.COLOR_BGR2GRAY)
img3R = cv2.cvtColor(img3R, cv2.COLOR_BGR2GRAY)

In [ ]:
D3L = match_SSD(img3L, img3R, 10, 100, True)
cv2.imwrite('ps2-5-a-1.png', scale_d(D3L))

In [153]:
img3L_blur = cv2.GaussianBlur(img3L, (11, 11), 9)
# img3L_sharpened = cv2.GaussianBlur()
cv2.imwrite("img3_blur.png", img3L_blur)

True

In [ ]:
D3L_blur = match_SSD(img3L_blur, img3R, 10, 100, True)
cv2.imwrite('ps2-5-a-2.png', scale_d(D3L_blur))

Answer:
